In [1]:
from bs4 import BeautifulSoup
import requests
import json
import os
import time

from csv import writer
from tqdm import tqdm, trange
from string import digits
from os.path import exists
from random import randrange

In [2]:
# set up proxies
useful_proxy = {}
i = 0

def get_proxy():
    proxies = [] # proxies poll
    proxy = {} # now using proxy
    
    # load the proxies poll
    proxies.clear()
    url='https://sslproxies.org/'
    response=requests.get(url)
    soup=BeautifulSoup(response.content, 'lxml')
    for item in soup.select('table[class="table table-striped table-bordered"] > tbody > tr'):
        try:
            proxies.append({'ip': item.select('td')[0].get_text(), 
                            'port': item.select('td')[1].get_text()})
        except:
            pass
    
    # random select a current proxy
#     rnd = randrange(len(proxies))
    global i
    if i >= len(proxies):
        i = 0
    randomIP = proxies[i]['ip']
    randomPort = proxies[i]['port']
    proxy =  {"https": f"https://{randomIP}:{randomPort}"}
    i += 1
    print(f"now using proxy {proxy}")
    return proxy
    
def repeat_get(source_url):
    global useful_proxy
    if useful_proxy:
        current_proxy = useful_proxy
    else:
        current_proxy = get_proxy()
    response = None
    while not response:
        try:
            response = requests.get(source_url, proxies = current_proxy).text
            if not useful_proxy and response: # useful_proxy is empty and response get
                useful_proxy = current_proxy
        except:
            print(f"discard current useful proxy {current_proxy}.")
            useful_proxy = {}
            current_proxy = get_proxy()
    return response

In [3]:
# test module
# response = repeat_get("https://en.wikipedia.org/wiki/Game_theory")
# soup = BeautifulSoup(response, 'html.parser')
# soup

## Tourist attractions in US cities

Source: https://www.city-data.com/articles/


In [4]:
class TouristSite:
    def __init__(self, name, state, title, description):
        # Init instance of national site
        self.name = name
        self.state = state
        self.title = title
        self.description = description
    
    def info(self):
        # Return a string representation of itself
        rep = '"' + '","'.join([self.name, self.state, self.title, self.description]) + '"\n'    
        return rep
            
    def write_single_csv(self):
        filename = './data/' + self.state + '_documents.csv'
        if not exists(filename):
            with open(filename, "x") as file:
                file.write("Name,State,Title,Description\n")
        with open(filename, "a", encoding='utf-8') as file:
            file.write(self.info())
            
    def write_whole_csv(self):
        filename = 'documents.csv'
        if not exists(filename):
            with open(filename, "x") as file:
                file.write("Name,State,Title,Description\n")
        with open(filename, "a", encoding='utf-8') as file:
            file.write(self.info())

In [5]:
def get_state_url_dict(source = "https://www.city-data.com/articles/"):
    # get a dict of state page name and its urls to state page
    response = response = requests.get(source).text
    soup = BeautifulSoup(response, 'html.parser')
    
    selector = 'div[id="tabs_by_category"] > ul[class="tab-list tab-list-short"] > li > a'
    state_urls = soup.select(selector)
    state_dict = {
        state_url['href'].split('/')[-1].split('.')[0]: \
            'https:' + state_url['href'] \
            for state_url in state_urls
    }
    return state_dict # key: state_page_name, value: url

In [6]:
def cache_sites_for_state_page(state_page_name, state_url):
    # cache the sites in a state page and store it into json file, like Florida6.json
    response = repeat_get(state_url)
    soup = BeautifulSoup(response, 'html.parser')
    
    # read the site pages
    selector = 'div[id="listing"] > div[class="col-md-4"] > a'
    site_urls = soup.select(selector)
    print(f'-> Reading site pages from state page <{state_page_name}>...')
    site_dict = {}
    for site_url in tqdm(site_urls):
        key = 'https://www.city-data.com' + site_url['href']
        value = repeat_get('https://www.city-data.com' + site_url['href'])
        site_dict[key] = value
#         time.sleep(1) # add a gap between two request to prevent failure

    # write to cache file
    print(f'-> Caching state page <{state_page_name}>...')
    filename = './cache/' + state_page_name + '.json'
    with open(filename, 'w') as json_file:
        json.dump(site_dict, json_file, indent=4)
    print('Done.')
    
    return site_urls # a list of site urls in state page
    
def get_cache_sites_for_state_page(state_page_name):
    # get a list of site urls given the state page
    filename = './cache/' + state_page_name + '.json'
    try:
        with open(filename) as json_file:
            site_dict = json.load(json_file)
    except:
        return None
    
    site_list = list(site_dict.keys())
    return site_list

In [7]:
def get_cache_site_page(state_page_name, site_url):
    # helper function: get response of a site page, return None if not exist
    filename = './cache/' + state_page_name + '.json'
    try:
        with open(filename) as json_file:
            site_dict = json.load(json_file)
            response = site_dict[site_url]
    except:
        return None
    
    return response

def get_site_instance(state_page_name, site_url):
    # get the site instance for a site page from a state_page_name and site_url
    response = get_cache_site_page(state_page_name, site_url)
    soup = BeautifulSoup(response, 'html.parser')
    
    state = state_page_name.translate(str.maketrans('', '', digits))
    title = soup.select('h1[class="city"] > span')[0].contents[0]
    paragraphs = [p.contents[0] for p in soup.select('div[class="well"] > p') if not p.find_all()]
    description = ' '.join(paragraphs)
    if title.count(' - ') == 2:
        name = title.split(' - ')[0]
    elif title.count(', ') == 2:
        name = title.split(', ')[0]
    else:
        name = title

    touristsite = TouristSite(name.replace('"', ""),
                              state.replace('"', ""), 
                              title.replace('"', ""), 
                              description.replace('"', ""))
    
    return touristsite

### Main

In [8]:
# cache html to local
state_dict = get_state_url_dict()
print("--- START CACHING FROM SOURCE WEBSITE ---")
for state_page_name, state_url in tqdm(state_dict.items()):
    filename = './cache/' + state_page_name + '.json'
    if state_page_name[-1].isdigit() and not exists(filename):
        cache_sites_for_state_page(state_page_name, state_url)
print("---> CACHING DONE!")

100%|███████████████████████████████████████████████████████████████████████████████| 77/77 [00:00<00:00, 38498.20it/s]

--- START CACHING FROM SOURCE WEBSITE ---
---> CACHING DONE!


In [9]:
# write to the csv file
print("--- START WRITING FILES TO DOC DATA ---")
for state_page_name, state_url in tqdm(state_dict.items()):
    site_list = get_cache_sites_for_state_page(state_page_name)
    print(f"Writing sites from <{state_page_name}> to csv file...")
    filename = './data/' + state_page_name + '_documents.csv'
    if not exists(filename):
        for site_url in tqdm(site_list):
            try:
                site = get_site_instance(state_page_name, site_url)
                site.write_single_csv()
                site.write_whole_csv()
            except:
                pass

  0%|                                                                                           | 0/77 [00:00<?, ?it/s]

--- START WRITING FILES TO DOC DATA ---



  0%|                                                                                          | 0/219 [00:00<?, ?it/s]

Writing sites from <Alaska> to csv file...



 30%|████████████████████████▍                                                        | 66/219 [00:30<01:16,  2.00it/s]


...

  0%|                                                                                          | 0/134 [00:00<?, ?it/s]

Writing sites from <Wyoming> to csv file...



 49%|███████████████████████████████████████▉                                         | 66/134 [00:17<00:18,  3.72it/s]


 99%|██████████████████████████████████████████████████████████████████████████████▊ | 132/134 [00:35<00:00,  3.67it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 77/77 [1:50:18<00:00, 85.96s/it]


In [10]:
# check exception
state_dict = get_state_url_dict()
exception_state_page_name_list = []
cachelines_dict = {}
datalines_dict = {}
for state_page_name, state_url in state_dict.items():
    print(f"Now doing page <{state_page_name}>")
    cachename = f"./cache/{state_page_name}.json"
    if state_page_name[-2].isdigit():
        data_page_name = state_page_name[:-2]
    elif state_page_name[-1].isdigit():
        data_page_name = state_page_name[:-1]
    else:
        data_page_name = state_page_name
    dataname = f"./data/{data_page_name}_documents.csv"
        
    with open(cachename) as json_file:
        site_dict = json.load(json_file)
        cachelines_dict[state_page_name] = len(site_dict)
    with open(dataname, encoding='utf-8') as f:
        datalines = sum(1 for line in f) - 1
        datalines_dict[data_page_name] = datalines

real_cachelines_dict = {}
for key, value in cachelines_dict.items():
    real_key = ''.join(i for i in key if not i.isdigit())
    if real_key in real_cachelines_dict.keys():
        real_cachelines_dict[real_key] += value
    else:
        real_cachelines_dict[real_key] = value

Now doing page <Alaska>
Now doing page <Alabama>
Now doing page <Arkansas>
Now doing page <Arizona>
Now doing page <Arizona2>
Now doing page <Arizona3>
Now doing page <California>
Now doing page <California2>
Now doing page <California3>
Now doing page <California4>
Now doing page <California5>
Now doing page <California6>
Now doing page <California7>
Now doing page <California8>
Now doing page <California9>
Now doing page <California10>
Now doing page <California11>
Now doing page <Colorado>
Now doing page <Colorado2>
Now doing page <Connecticut>
Now doing page <District-of-Columbia>
Now doing page <Delaware>
Now doing page <Florida>
Now doing page <Florida2>
Now doing page <Florida3>
Now doing page <Florida4>
Now doing page <Florida5>
Now doing page <Florida6>
Now doing page <Georgia>
Now doing page <Georgia2>
Now doing page <Hawaii>
Now doing page <Hawaii2>
Now doing page <Hawaii3>
Now doing page <Iowa>
Now doing page <Idaho>
Now doing page <Illinois>
Now doing page <Indiana>
Now do

In [13]:
import pandas as pd
pd.DataFrame([real_cachelines_dict, datalines_dict]).T

,0,1
Alaska,219,220
Alabama,130,130
Arkansas,165,165
Arizona,756,756
California,3094,3092
Colorado,459,459
Connecticut,197,197
District-of-Columbia,108,108
Delaware,121,121
Florida,1730,1730
